In [22]:
import json
from typing import List, Dict
from src.llms import LLM
from src.prompts import *
from src.llms import OpenAILLM

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=32)
import json
import random
import pandas as pd

%load_ext autoreload
%autoreload 2


INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = pd.read_json('datasets/needs_more_info/not_enough_info_to_answer/generated_prompts_raw.json')


In [7]:
df[0][0]

"I'm in a legal dispute with my neighbor over property boundaries. What's the best course of action?"

In [26]:
import src.prompts
system_prompt = "You are a helpful assistant designed to deliver useful evaluations. You respond only with yes or no, and nothing else."
def get_relevance_score(question):
    relevance_prompt = generate_revelance_prompt(question)
    LLM = OpenAILLM
    llm = LLM("gpt-3.5-turbo",system_prompt)
    response = llm.chat(relevance_prompt, logprobs=True)
    try:
        # Extract the log probabilities from the response
        logprobs = response['choices'][0]['logprobs']['token_logprobs']
        
        # Calculate the relevance score based on log probabilities
        # This is a simplified example; you may need to adjust based on your specific use case
        relevance_score = sum(logprobs) / len(logprobs)
        return relevance_score
    except Exception as e:
        print(f"Error extracting relevance score: {e}")
        return None



# Sample 10 prompts from the 'text' column of the DataFrame
sample_prompts = df[0].sample(n=10, random_state=42)

# Calculate relevance scores for the sampled prompts
relevance_scores = []
for prompt in sample_prompts:
    score = get_relevance_score(prompt)
    relevance_scores.append((prompt, score))

# Print the results
print("Relevance scores for 10 sampled prompts:")
for prompt, score in relevance_scores:
    print(f"Prompt: {prompt}")
    print(f"Relevance Score: {score}")
    print("-" * 50)


TypeError: OpenAILLM.chat() got an unexpected keyword argument 'logprobs'

In [69]:
question = "i am confused"
relevance_prompt = generate_revelance_prompt(question)
LLM = OpenAILLM
llm = LLM("gpt-3.5-turbo",system_prompt)
response = llm.chat(relevance_prompt, True)


In [70]:

response["logprobs"]

ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='No', bytes=[78, 111], logprob=-3.8338785e-05, top_logprobs=[TopLogprob(token='No', bytes=[78, 111], logprob=-3.8338785e-05), TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-10.257484), TopLogprob(token='This', bytes=[84, 104, 105, 115], logprob=-13.984957), TopLogprob(token='I', bytes=[73], logprob=-14.64805), TopLogprob(token=' No', bytes=[32, 78, 111], logprob=-15.06167)])])

In [55]:
from openai import OpenAI
import os

model = "gpt-3.5-turbo"
messages = [{"role": "system", "content": system_prompt}]

client = OpenAI( api_key= os.getenv("OPENAI_API_KEY"))

logprobs = True
response = client.chat.completions.create(
            messages=messages, 
            model=model,
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            logprobs=logprobs,
            top_logprobs=5 if logprobs else None
        )
response_text = response.choices[0].message.content


# # Extract and print log probabilities
# logprobs = response.choices[0].logprobs
# if logprobs:
#     print("Log probabilities:")
#     for token, prob in zip(logprobs.tokens, logprobs.token_logprobs):
#         print(f"Token: {token}, Log Probability: {prob}")
# else:
#     print("Log probabilities not available")

In [56]:
logprobs_data = response.choices[0].logprobs

In [41]:
# Display logprobs data in a more readable format
print("Log Probabilities Data:")
print("-----------------------")
for token_logprob in logprobs_data.content:
    print(f"Token: {token_logprob.token}")
    print(f"  Log Probability: {token_logprob.logprob:.4f}")
    print("  Top Logprobs:")
    for top_logprob in token_logprob.top_logprobs:
        print(f"    {top_logprob.token}: {top_logprob.logprob:.4f}")
    print()


Log Probabilities Data:
-----------------------
Token: Under
  Log Probability: -0.4528
  Top Logprobs:
    Under: -0.4528
    Got: -1.6928
    Yes: -1.8707
    Great: -3.9551
    Sure: -5.5326

Token: stood
  Log Probability: -0.0004
  Top Logprobs:
    stood: -0.0004
    stand: -7.7487
    sto: -13.6454
     the: -15.3768
    stable: -15.7495

Token: !
  Log Probability: -1.3977
  Top Logprobs:
    .: -0.4985
    !: -1.3977
    ?: -2.0476
    ,: -4.3852
    <|end|>: -5.8374

Token:  Let
  Log Probability: -1.7647
  Top Logprobs:
     How: -0.8145
     Feel: -1.6937
     Let: -1.7647
     Just: -2.8366
     Please: -3.0590

Token: 's
  Log Probability: -0.3057
  Top Logprobs:
    's: -0.3057
     me: -1.3352
    ’s: -8.2831
     us: -10.9250
     the: -11.2707

Token:  get
  Log Probability: -0.0433
  Top Logprobs:
     get: -0.0433
     begin: -3.6463
     start: -4.1875
     proceed: -6.9928
     do: -9.3464

Token:  started
  Log Probability: -0.0000
  Top Logprobs:
     started: -